In [ ]:
%pip install -e ../../

In [ ]:
"""
Example of using the SigKit library to create a signal, apply an impairment,
calculate metrics and display visuals.
"""
import numpy as np

from sigkit.core.base import Signal
from sigkit.modem.psk import PSK
from sigkit.impairments.awgn import AWGN
from sigkit.metrics.integrity import estimate_snr, calculate_ber
from sigkit.metrics.visuals import plot_constellation, plot_spectrogram, plot_psd, plot_time, plot_frequency

In [ ]:
"""Create a Modulated Signal from Bits"""

sample_rate = 1024 # Sample rate in Hz
symbol_rate = 32 # Symbol rate
carrier_frequency = 0 # Carrier frequency in Hz, 0 is baseband

modem = PSK(
    sample_rate=sample_rate,
    symbol_rate=symbol_rate,
    n_components=4, # Number of PSK components (e.g., 4 for QPSK)
    cf=carrier_frequency
)

num_symbols = 64  # Number of symbols to modulate
bits = np.random.randint(
    0, 2, size=64 * modem.bits_per_symbol, dtype=np.uint8
)
signal: Signal = modem.modulate(bits)

# Signal members are set with the attributes of the modem
print(signal, end="\n\n")
print(f"{signal.sample_rate=}")
print(f"{signal.carrier_frequency=}")


In [ ]:
"""Apply an Impairment"""
impairment = AWGN(snr_db=20)  # Target Signal to Noise Ratio in dB
signal: Signal = impairment(signal) # Impairments return a new Signal object for immutability

In [ ]:
"""Calculate Metrics"""
demod_bits = modem.demodulate(signal.samples)  # Demodulate the signal to bits

ber = calculate_ber(demod_bits, truth_bits=bits)  # Bit Error Rate
snr = estimate_snr(noisy=demod_bits, clean=bits)  # Signal to Noise Ratio

print(f"{ber=}")
print(f"{snr=}")

In [ ]:
"""Display Visuals"""
plot_constellation(signal)  # Plot the constellation diagram
plot_spectrogram(signal)  # Plot the spectrogram of the signal
plot_psd(signal)  # Plot the Power Spectral Density of the signal
plot_time(signal)  # Plot the time domain representation of the signal
plot_frequency(signal)  # Plot the frequency domain representation of the signal